<a href="https://colab.research.google.com/github/Fatih120/GuildedChatExporter/blob/main/guildedchatexporter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GuildedChatExporter v2

Export your stuff before it's gone.
Full Transparency : 80% done with AI.

Todo: cleanup, less crap code, HTML forums and output other at least in txt (unlikely)

Need help?
https://github.com/Fatih120/GuildedChatExporter/tree/main


[Mountain of Fatih Guilded](https://guilded.gg/MoF)

[Discord :(](https://discord.com/invite/Cy27FNfQtc)

IF THERE ARE ANY ISSUES PLEASE DON'T HESITATE TO CONTACT.



In [ ]:
# @title Basic Setup

# @markdown 1.   Log on to Guilded on your web browser, not client, and focus on that tab.

# @markdown 2.   If based on **Firefox**, press `SHIFT + F9`.
# @markdown
# @markdown If on ***Chrome***, press `F12` and find the "Application" tab.
# @markdown
# @markdown We're here to grab your login, so click the Cookies entry on the sidebar, and then the guilded.gg url.
# @markdown
# @markdown Find the `hmac_signed_session` key at the bottom. Copy the value, it is very long. Paste it here.

#!apt-get install -y p7zip-full
import requests
import json
import re
import os
import shutil
import ipywidgets as widgets
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse, unquote
from google.colab import files
!pip install emoji
import emoji

USER_TOKEN = "4c138f5590c592d0da0199e643204fb2e2cc26fee4c1785aad584297bb33feb01ec2a6139b1d8972b3bdf6974c9723.1010149fcc020fd6166b2e873abcac03.87bc26dad6b753983a3d958c13f0ed973c9de72af0a5bd6d28f6078739953974" # @param {type:"string"}
# @markdown Do NOT share this value with anyone as they will be able to access your account without a password.

SAVE_DIRECTORY = "/guildedchatexporter/" # @param {type:"string"}
if not os.path.exists(SAVE_DIRECTORY):
  os.makedirs(SAVE_DIRECTORY)

cookies = {
    "authenticated": "true",
    "hmac_signed_session": USER_TOKEN,
}


# @markdown Google Colab gives you over 100GB of temporary space to work with, which should be way more than enough to store and then download your stuff.
# @markdown
# @markdown  You can also connect your Google Drive instead and copy the stuff to there at the end, though if you just want to download your files immediately without using Google Drive, skip the optional step.
# --------------------------------------------------- #

guilded = "https://www.guilded.gg/api"
def fetch(endpoint, params=None):
    url = f"{guilded}/{endpoint}"
    response = requests.get(url, params=params, cookies=cookies)
    if response.status_code == 200:
      return unshid_cdn(response.json())
    else:
        raise Exception(f"Error fetching data: {response.status_code} {response.text}")
        print("Did you correctly input your token?")

def unshid_cdn(data):
  def fix_url(url):
    match = re.match(r"^https://s3-us-west-2\.amazonaws\.com/www\.guilded\.gg/(.*)$", url)
    if match:
      return f"https://cdn.gilcdn.com/{match.group(1)}"
    else:
      return url

  def fix_dict(d):
    for key, value in d.items():
      if isinstance(value, dict):
        fix_dict(value)
      elif isinstance(value, list):
        for item in value:
          if isinstance(item, dict):
            fix_dict(item)
          elif isinstance(item, str):
            d[key] = fix_url(item)
      elif isinstance(value, str):
        d[key] = fix_url(value)

  fix_dict(data)
  return data

def fetch_user(user_id):
    data = fetch(f"users/{user_id}")
    return data["user"]

def fetch_channel(channel_id):
    data = fetch("content/route/metadata", params={"route": f"//channels/{channel_id}/chat"})
    return data["metadata"]["channel"]

def fetch_servers():
    data = fetch("me", params={"isLogin": "false", "v2": "true"})
    return data['teams']

def fetch_members():
    data = fetch(f"teams/{SERVER_ID}/members")
    return data

def get_groups():
    return fetch(f"teams/{SERVER_ID}/groups")

def get_groups():
    return fetch(f"teams/{SERVER_ID}/groups")

def get_channels():
    return fetch(f"teams/{SERVER_ID}/channels")

def fetch_dms():
    return fetch(f"users/{USER_ID}/channels")

def fetch_myself():
    return fetch("me", params={"isLogin": "false", "v2": "true"})

def fetch_info():
    return fetch(f"teams/{SERVER_ID}/info")

def sanitize_filename(filename):
    # Replace invalid characters with underscores
    filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
    return filename[:255]

from IPython.display import clear_output
clear_output()

print("Please select the server you wish to archive in the dropdown.")
server_dict = {f"{server['name']} ({server['id']})": server["id"] for server in fetch_servers()}
server_dict[''] = None
server_dropdown = widgets.Dropdown(options=list(server_dict.keys()), value='', description='Servers:',)
display(server_dropdown)

def on_server_select(change):
    global SERVER_ID
    global SERVER_NAME
    selected_server_name = change["new"]
    SERVER_ID = server_dict[selected_server_name]
    SERVER_NAME = selected_server_name
    print(f"Selected server: {selected_server_name}")
    print("You can go to the next step, or you can go straight for DMs")

server_dropdown.observe(on_server_select, names="value")

In [ ]:
# @title (Optional) Connect Google Drive
save_to_gdrive = False # @param {type:"boolean"}
from google.colab import drive
drive.mount('/content/drive')
if save_to_gdrive:
    SAVE_DIRECTORY = "/content/drive/MyDrive/guildedchatexporter/"
print("Done, do next step")

In [ ]:
# @title Save Basic Data about Server and Users
if 'SERVER_ID' not in globals():
    print("You forgot to choose a server. Go back and select one from the dropdown.")
else:
    server_dir = os.path.join(SAVE_DIRECTORY, SERVER_NAME)
    os.makedirs(server_dir, exist_ok=True)

verbose = False # @param {type:"boolean"}
get_audits = False # @param {type:"boolean"}
# @markdown You can try to get audit logs for the server, but it'll fail if you're not authorized to see them.

def download_file(url, filepath):
    !wget -q -nc -O "$filepath" "$url"
mc = 0
members_data = fetch_members()
members = members_data.get('members', [])

overview = fetch(f"teams/{SERVER_ID}/overview")
with open(os.path.join(server_dir, 'overview.json'), 'w') as f:
    f.write(json.dumps(overview, indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/overview.json")


all_audit_logs = []
before_date = None
if get_audits == True:
    print("Getting audit logs...")
    while True:
        params = {"maxItems": 50}
        if before_date:
            params["beforeDate"] = before_date
        response = requests.post(f"https://www.guilded.gg/api/teams/{SERVER_ID}/auditlogs", json=params, cookies=cookies)
        response.raise_for_status()
        audit_logs = response.json()
        new_logs = audit_logs["logs"]
        all_audit_logs.extend(new_logs)
        if len(new_logs) < 50:
            break
        !sleep 1
        before_date = new_logs[-1]["createdAt"]
    with open(os.path.join(server_dir, 'auditlogs.json'), 'w') as f:
        json.dump(all_audit_logs, f, indent=4)
    print(f"Saved {len(all_audit_logs)} audit logs to {os.path.join(server_dir, 'auditlogs.json')}")



for member in members:
    member_id = member["id"]
    member_name = member["name"]
    member_dir = os.path.join(server_dir, "members", member_name)
    os.makedirs(member_dir, exist_ok=True)
    user_data = fetch_user(member_id)

    with open(os.path.join(member_dir, f"{member_id}.json"), 'w') as f:
        f.write(json.dumps(user_data, indent=4))
    json_string = json.dumps(user_data)

    cdn_links = re.findall(r"https://cdn\.gilcdn\.com/[^\"]+", json_string)
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(download_file, url, os.path.join(member_dir, url.split("/")[-1].split("?")[0])) for url in cdn_links]

    mc += 1
    if verbose:
      print(f"{str(mc)}/" + str(len(members)) + f" {member_name} ({member_id})")



with open(os.path.join(server_dir, 'members.json'), 'w') as f:
    f.write(json.dumps(fetch_members(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/members.json")
with open(os.path.join(server_dir, 'info.json'), 'w') as f:
    infourls = []
    info = fetch_info()
    f.write(json.dumps(info, indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/info.json")
    if 'profilePicture' in info['team']:
        infourls.append(info['team']['profilePicture'])
    if 'teamDashImage' in info['team']:
        infourls.append(info['team']['teamDashImage'])
    if 'homeBannerImageLg' not in info['team']:
        if 'homeBannerImageMd' in info['team']:
            infourls.append(info['team']['homeBannerImageMd'])
        if 'homeBannerImageSm' in info['team']:
            infourls.append(info['team']['homeBannerImageSm'])
    else:
        infourls.append(info['team']['homeBannerImageLg'])
    for url in infourls:
        if verbose:
            print(f"checking: {type(url)}, {url}")
        if isinstance(url, bytes):
            print(f"this is apparently a BYTE URL: {url}")
            continue

        try:
            parsed_url = urlparse(url)
            basename = os.path.basename(parsed_url.path)
            full_path = os.path.join(server_dir, basename)
            if verbose:
                print(f"Basename: {type(basename)}, {basename}")
                print(f"Full path: {type(full_path)}, {full_path}")
            download_file(url, full_path)
            print(f"Successful: {url}")
        except Exception as e:
            print(f"Error with URL {url}: {str(e)}")

with open(os.path.join(server_dir, 'groups.json'), 'w') as f:
    f.write(json.dumps(get_groups(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/groups.json")
with open(os.path.join(server_dir, 'channels.json'), 'w') as f:
    f.write(json.dumps(get_channels(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/channels.json")

print("Done getting some fundamental stuff, go to the next step")

In [ ]:
# @title Save All Emoji
from google.colab import files
emotes_data = fetch(f"teams/{SERVER_ID}/customReactions")

verbose = False # @param {type:"boolean"}
emotes_dir = os.path.join(server_dir, "emotes")
download_immediately = False # @param {type:"boolean"}

def fetch_emotes():
    os.makedirs(emotes_dir, exist_ok=True)

    with open(os.path.join(server_dir, 'emotes.json'), 'w') as f:
        f.write(json.dumps(emotes_data, separators=(',', ':')))
    print("Saved emotes.json")

    ec = 0
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        downloaded = set()  # need to check if we already got it cuz dupes, waste less time
        for reaction in emotes_data.get('reactions', []):
            for image_type in ["png", "webp", "apng"]:
                url = reaction.get(image_type)
                if url and url not in downloaded:
                    filename = url.split("/")[-1].split("?")[0]
                    filepath = os.path.join(emotes_dir, reaction["name"] + "." + filename.split(".")[-1])
                    futures.append(executor.submit(download_emote, url, filepath))
                    downloaded.add(url)
                    ec += 1
                    if verbose:
                      print(f"{ec}/{str(len(emotes_data.get('reactions', [])))} " + reaction["name"])

        for future in as_completed(futures):
            future.result()

        print("All done with server emotes!")

def download_emote(url, filepath):
    !wget -q -nc -O "$filepath" "$url"

fetch_emotes()


# This is so tragic omg.

role_icons_dir = os.path.join(emotes_dir, "roleIcons")
os.makedirs(role_icons_dir, exist_ok=True)
with open(os.path.join(server_dir, "info.json"), "r") as f:
    info_data = json.load(f)
os.makedirs(role_icons_dir, exist_ok=True)
print(f"Getting external emoji")

for role_data in info_data.get("team", {}).get("rolesById", {}).values():
    icon_url = role_data.get("iconUrl")
    if icon_url:
        if icon_url.startswith("http"): # Check if it looks like a valid URL
            filename = os.path.basename(urlparse(icon_url).path)
            download_path = os.path.join(role_icons_dir, filename)
            with open(download_path, "wb") as f:
                f.write(requests.get(icon_url).content)

if download_immediately:
    emojizip = os.path.join(SAVE_DIRECTORY, f"{SERVER_NAME}_emoji.zip")
    shutil.make_archive(emojizip[:-4], 'zip', emotes_dir)
    files.download(emojizip)

print("Done grabbing emoji, go to next step")

In [ ]:
# @title HTML File Setup
# @markdown Has the base HTML and CSS information which you can edit. Just click RUN to set the info for the next step.
# @markdown
## HTML Template

CSS = """
@font-face {
  font-family: "Builder Sans";
  src: url("https://www.guilded.gg/fonts/BuilderSans-Regular.woff2") format('woff2');
}

body {
  font-family: 'Builder Sans', Tahoma, sans-serif;
  color: white;
  background-color: #373943;
}

#chat-log {
  background-color: #373943;
  padding: 16px;
  display: flex;
  flex-direction: column;
  gap: 8px;
}

.message {
  color: #ececee;
  font-size: 15px;
  line-height: 145%;
  font-weight: normal;
  font-family: 'Builder Sans', Tahoma, sans-serif;
  margin-top: 0px;
  min-height: 44px;
  padding: 8px 12px;
  display: flex;
  align-content: flex-start;
  gap: 8px;
  transition: background-color 140ms ease;
  position: relative;
  contain: layout;
  max-width: 80%;
}

.message:hover {
  background-color: #31333c;
}

.created-at {
  margin-right: 8px;
  height: 11px;
  color: #a3a3ac;
  font-size: 13px;
  line-height: 120%;
  font-weight: normal;
  white-space: nowrap;
  pointer-events: auto;
}

.created-by {
  font-size: 15px;
  line-height: 145%;
  font-weight: 700;
  white-space: nowrap;
  pointer-events: auto;
}

.pfp {
  width: 40px;
  height: 40px;
  border-radius: 50%;
  overflow: hidden;
  vertical-align: middle;
  flex-shrink: 0;
  margin-right: 12px;
  pointer-events: auto;
}

.content {
  margin: 0;
  word-break: break-all;
}

.content img,
.content video {
  max-height: 320px;
  max-width: 360px;
  object-fit: contain;
}

.file-upload-container {
  display: flex;
  align-items: center;
  background-color: #32343d;
  padding: 10px;
  border: 1px #a3a3ac solid;
  border-radius: 5px;
	color: #fef0b9;
}

.file-icon {
  margin-right: 10px;
	background-image: url('{icon-file-attach.svg');
  background-repeat: no-repeat;
  background-size: contain;
}

.file-info {
    flex-grow: 1;
}

.file-name {
    font-weight: bold;
    text-decoration: none;
    color: #fef0b9;
}

.file-size {
    color: #666;
    font-size: 0.9em;
}

.download-icon {
    margin-left: 10px;
    color: #666;
}

.icon {
    width: 20px;
    height: 20px;
}

"""

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>#{title}</title>
    <style>
        {CSS}
    </style>
</head>
<body>
    <div id="chat-log">
        <p>#{title}</p>
        <p>{topic}</p>
        <hr>
        {messages}
    </div>
</body>
</html>
"""

## Message Template
MESSAGE_TEMPLATE = """
<div class="message">
    <img class="pfp" src="{avatar_url}">
    <div>
        <span class="created-by">{created_by}</span>
        <span class="created-at">({created_at})</span>
        <p class="content">{content}</p>
    </div>
</div>
"""

print("HTML ready to print")


# Servers

In [ ]:
# @title Download all Media and Save HTMLs for every channel
messages_per_page = 10000 # @param {type:"integer"}
# @markdown This is the breadwinner - we will finally save and print everything to HTML files. It's huge and lengthy, so if you have an especially big server on top of that, you will probably want to leave and get a coffee while this runs.

# @markdown This will also split HTML files by the amount of messages, because you really don't wanna open a file over 10Mb large in your browser. 10000 should be a nice default and still might be too laggy depending on your channel, so increase it or decrease it as you need. Nothing will happen if your channel has less messages than this.

verbose_mescount = False # @param {type:"boolean"}

# @markdown WIP - Most other channels are only grabbed in raw JSON minimized, will probably try to work on forums but no promises (i also haven't readily downloaded media from forums and docs just yet, forgive my lack of time)

# Load emotes data
with open(os.path.join(server_dir, "emotes.json"), "r") as f:
    emotes_data = json.load(f)
    emotes_dict = {str(emote['id']): emote for emote in emotes_data.get('reactions', [])}
# Load server info for roles
with open(os.path.join(server_dir, "info.json"), "r") as f:
    server_info = json.load(f)
    roles_iterable = server_info['team']['rolesById'].values() if isinstance(server_info['team']['rolesById'], dict) else server_info['team']['rolesById']
    roles_data = {str(role['id']): role for role in roles_iterable}
# Load members data
with open(os.path.join(server_dir, "members.json"), "r") as f:
    members_json = json.load(f)
    members_dict = {member['id']: member for member in members_json['members']}
# Load channels data
with open(os.path.join(server_dir, "channels.json"), "r") as f:
    channels_data = json.load(f)
with open(os.path.join(server_dir, "groups.json"), "r") as f:
    groups_data = json.load(f)

def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "wb") as f:
            f.write(response.content)
    else:
        print(f"Error fetching data from '{url}': Status code {response.status_code}")
        return None

def get_colored_name(member_id, name):
    def get_member_color(member):
        if member and 'roleIds' in member:
            for role_id in member['roleIds']:
                role = roles_data.get(str(role_id))
                if role and role['color'] != 'transparent':
                    return role['color']
        return None

    member = next((m for m in members_data['members'] if m['id'] == member_id), None)
    color = get_member_color(member)

    if color:
        return f'<span style="color: {color};">@{name}</span>'
    return f'@{name}'

def generate_html(messages, channel_data, members_data, server_dir, channel_dir):
    formatted_messages = []
    doge = {}

    with ThreadPoolExecutor() as executor:
        futures = []

        for message in messages:
            created_by_id = message['createdBy']
            created_at = message.get('createdAt') or message.get('created_at')

            # Determine if it's a system message
            is_system_message = message.get('type') == 'system' or 'systemMessage' in str(message.get('content', {}))

            if is_system_message:
                member = next((m for m in members_data['members'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"
                avatar_url = "https://www.guilded.gg/asset/DefaultUserAvatars/profile_5.png"

                # Handle system message content
                content = ""
                document = message.get('content', {}).get('document', {})
                for node in document.get('nodes', []):
                    if node.get('type') == 'systemMessage':
                        data = node.get('data', {})
                        message_type = data.get('type')
                        if message_type == 'team-channel-created':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) created this chat channel."
                        elif message_type == 'channel-renamed':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) renamed this channel from '{data.get('oldName')}' to '{data.get('newName')}'."
                        elif message_type == 'streaming-screenshare-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started to share their screen."
                        elif message_type == 'voice-call-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started a voice call."
                        elif message_type == 'webhookMessage':
                            embeds = node.get('data', {}).get('embeds', [])
                            content = json.dumps(data, indent=2)
                        else:
                            content = f"System action performed: {message_type}"
            else:
                # Find member data based on createdBy ID
                member = next((m for m in members_data['members'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"

                # Get the base role
                base_role = next((role for role in roles_data.values() if role.get('isBase')), None)
                base_color = base_role["color"] if base_role else None

                # Get the highest priority role for the member
                highest_priority_role = None
                if member and 'roleIds' in member:
                    member_roles = [roles_data.get(str(role_id)) for role_id in member["roleIds"] if str(role_id) in roles_data]
                    member_roles.sort(key=lambda x: x["priority"], reverse=True)
                    if member_roles:
                        highest_priority_role = member_roles[0]

                # Color the created_by name based on the highest priority role or base role
                if highest_priority_role and highest_priority_role["color"] != "transparent":
                    color = highest_priority_role["color"]
                elif base_role:
                    color = base_role["color"]
                else:
                    color = None

                if color:
                    created_by = f'<span style="color: {color};">{created_by}</span>'
                else:
                    created_by = created_by


                # Handle avatar URL
                if created_by_id in members_dict and members_dict[created_by_id].get('profilePicture'):
                    avatar_url = os.path.join("..", "..", "members", members_dict[created_by_id].get('name'), os.path.basename(members_dict[created_by_id].get('profilePicture')).split('?')[0])
                else:
                    if created_by_id not in doge:
                        avatar_number = (len(doge) % 5) + 1
                        doge[created_by_id] = avatar_number
                    avatar_url = f"https://www.guilded.gg/asset/DefaultUserAvatars/profile_{doge[created_by_id]}.png"

                # Process regular message content
                content = ""
                if "content" in message and message["content"]:
                    document = message["content"].get("document")
                    if document:
                        nodes = document.get("nodes", [])
                        for node in nodes:
                            if node.get("type") in ("image", "video", "fileUpload"):
                                file_src = node.get("data", {}).get("src", "")
                                if file_src:
                                    parsed_url = urlparse(file_src)
                                    media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                    if node.get("type") == "fileUpload":
                                        filename = node.get("data", {}).get("name")
                                        folder = "files"
                                    elif node.get("type") == "image":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "images"
                                    elif node.get("type") == "video":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "videos"

                                    filepath = os.path.join(channel_dir, "media", folder, filename)
                                    futures.append(executor.submit(download_file, media_url, filepath))

                                    if node.get("type") == "image":
                                        content += f'<img src="{os.path.join("media", "images", filename)}">'
                                    elif node.get("type") == "video":
                                        content += f'<video controls><source src="{os.path.join("media", "videos", filename)}" type="video/webm">Your browser does not support the video tag.</video>'
                                    elif node.get("type") == "fileUpload":
                                        file_size_bytes = node.get("data", {}).get("fileSizeBytes")
                                        file_size_kb = round(file_size_bytes / 1024, 1)
                                        content += f'''
                                            <div class="file-upload-container">
                                                <div class="file-icon">
                                                    <svg class="icon icon-file-attach" shape-rendering="geometricPrecision" role="img">
                                                        <use xml:space="http://www.w3.org/1999/xlink" xlink:href="#icon-file-attach"></use>
                                                    </svg>
                                                </div>
                                                <div class="file-info">
                                                    <a href="{os.path.join("media", "files", filename)}" download class="file-name">{filename}</a>
                                                    <div class="file-size">{file_size_kb} kb</div>
                                                </div>
                                                <a href="{os.path.join("media", "files", filename)}" download class="download-icon">
                                                    <svg class="icon icon-download-tray" shape-rendering="geometricPrecision" role="img">
                                                        <use xml:space="http://www.w3.org/1999/xlink" xlink:href="#icon-download-tray"></use>
                                                    </svg>
                                                </a>
                                            </div>
                                        '''
                                    content += '<br>'


                            if node.get("type") == "code-container":
                                code_lines = node.get("nodes", [])
                                code_content = ''
                                for code_line in code_lines:
                                    if code_line.get("type") == "code-line":
                                        code_line_nodes = code_line.get("nodes", [])
                                        for code_line_node in code_line_nodes:
                                            leaves = code_line_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                code_content += leaf_text + '\n'
                                language = node.get("data", {}).get("language", "")
                                if language:
                                    content += f'<pre><code class="language-{language}">{code_content.strip()}</code></pre>'
                                else:
                                    content += f'<pre><code>{code_content.strip()}</code></pre>'


                            elif node.get("type") == "paragraph":
                                sub_nodes = node.get("nodes", [])
                                for sub_node in sub_nodes:
                                    if sub_node.get("type") == "reaction":
                                        reaction_data = sub_node.get("data", {})
                                        if reaction_data:
                                            reaction = reaction_data.get("reaction")
                                            if reaction:
                                                reaction_id = reaction.get("id")
                                                if reaction_id:
                                                    if str(reaction_id).startswith("9000"):
                                                        reaction_name = reaction.get("name")
                                                        if reaction_name:
                                                            reaction_emoji = emoji.emojize(f":{reaction_name}:", language='alias')
                                                            content += reaction_emoji
                                                    else:
                                                        emoji_dict = next((item for item in emotes_dict.values() if item['id'] == reaction_id), None)
                                                        if emoji_dict:
                                                            content += f'<img src="{emoji_dict["webp"]}" alt="{emoji_dict["name"]}" title="{emoji_dict["name"]}" style="height: 1.3em; vertical-align: middle;">'
                                                        else:
                                                            custom_reaction = reaction.get("customReaction")
                                                            if isinstance(custom_reaction, bool):
                                                                custom_reaction = None
                                                            if custom_reaction and "webp" in custom_reaction:
                                                                emote_url = custom_reaction["webp"]
                                                                emote_filename = custom_reaction["name"]
                                                                emotes_external_dir = os.path.join(server_dir, "emotes", "external")
                                                                if not os.path.exists(emotes_external_dir):
                                                                    os.makedirs(emotes_external_dir)
                                                                emote_filepath = os.path.join(emotes_external_dir, emote_filename + ".webp")
                                                                if not os.path.exists(emote_filepath):
                                                                    response = requests.get(emote_url)
                                                                    if response.status_code == 200:
                                                                        with open(emote_filepath, "wb") as f:
                                                                            f.write(response.content)
                                                                content += f'<img src="../../emotes/external/{emote_filename}.webp" alt="{emote_filename}.webp" title="{emote_filename}.webp" style="height: 1.3em; vertical-align: middle;">'





                                    elif sub_node.get("type") == "mention":
                                        mention_data = sub_node.get("data", {}).get("mention", {})
                                        mentioned_id = mention_data.get("id")
                                        mentioned_name = mention_data.get("name")
                                        member = next((m for m in members_data['members'] if m['id'] == mentioned_id), None)
                                        current_name = member['name'] if member else mentioned_name
                                        display_name = f'@{mentioned_name}'
                                        content += display_name




                                    #elif sub_node.get("type") == "mention":
                                    #    mention_data = sub_node.get("data", {}).get("mention", {})
                                    #    mentioned_id = mention_data.get("id")
                                    #    mentioned_name = mention_data.get("name")
                                    #    member = next((m for m in members_data['members'] if m['id'] == mentioned_id), None)
                                    #    current_name = member['name'] if member else mentioned_name
                                    #    color = None
                                    #    if member and 'roleIds' in member:
                                    #        for role_id in member['roleIds']:
                                    #            role = roles_data.get(str(role_id))
                                    #            if role and role['color'] != 'transparent':
                                    #                color = role['color']
                                    #                break
                                    #    display_name = f'@{mentioned_name}'
                                    #    if current_name != mentioned_name:
                                    #        realname = f'Current name: @{current_name}'
                                    #        if color:
                                    #            content += f'<span style="color: {color};" title="{realname}">{display_name}</span>'
                                    #        else:
                                    #            content += f'<span title="{realname}">{display_name}</span>'
                                    #    else:
                                    #        if color:
                                    #            content += f'<span style="color: {color};">{realname}</span>'
                                    #        else:
                                    #            content += display_name

                                    elif sub_node.get("type") == "link":
                                        link_data = sub_node.get("data", {})
                                        href = link_data.get("href", "")
                                        if href:
                                            link_text = sub_node.get("nodes", [{}])[0].get("leaves", [{}])[0].get("text", "")
                                            content += f'<a href="{href}">{link_text}</a>'

                                    elif sub_node.get("type") == "block-quote-container":
                                        block_quote_lines = sub_node.get("nodes", [])
                                        block_quote_content = ''
                                        for block_quote_line in block_quote_lines:
                                            if block_quote_line.get("type") == "block-quote-line":
                                                block_quote_line_nodes = block_quote_line.get("nodes", [])
                                                for block_quote_line_node in block_quote_line_nodes:
                                                    leaves = block_quote_line_node.get("leaves", [])
                                                    for leaf in leaves:
                                                        leaf_text = leaf.get("text", "")
                                                        marks = leaf.get("marks", [])
                                                        for mark in marks:
                                                            mark_type = mark.get("type")
                                                            if mark_type == "inline-code-v2":
                                                                leaf_text = f'<code>{leaf_text}</code>'
                                                            elif mark_type == "bold":
                                                                leaf_text = f'<strong>{leaf_text}</strong>'
                                                            elif mark_type == "italic":
                                                                leaf_text = f'<em>{leaf_text}</em>'
                                                            elif mark_type == "underline":
                                                                leaf_text = f'<u>{leaf_text}</u>'
                                                            elif mark_type == "strikethrough":
                                                                leaf_text = f'<del>{leaf_text}</del>'
                                                            elif mark_type == "spoiler":
                                                                leaf_text = f'<span style="background-color: #000; color: #fff;">{leaf_text}</span>'
                                                        block_quote_content += leaf_text
                                                block_quote_content += '<br>'
                                        content += f'<div style="background-color: #f0f0f0; border-left: 4px solid #F5C400; padding: 10px;">{block_quote_content}</div>'

                                    else:
                                        leaves = sub_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            marks = leaf.get("marks", [])
                                            for mark in marks:
                                                mark_type = mark.get("type")
                                                if mark_type == "inline-code-v2":
                                                    if mark.get("object") == "mark":
                                                        leaf_text = sub_node.get("leaves", [{}])[0].get("text", "")
                                                        leaf_text = leaf_text.replace("<script>", "").replace("</script>", "")
                                                    leaf_text = f'<code>{leaf_text}</code>'
                                                elif mark_type == "bold":
                                                    leaf_text = f'<strong>{leaf_text}</strong>'
                                                elif mark_type == "italic":
                                                    leaf_text = f'<em>{leaf_text}</em>'
                                                elif mark_type == "underline":
                                                    leaf_text = f'<u>{leaf_text}</u>'
                                                elif mark_type == "strikethrough":
                                                    leaf_text = f'<del>{leaf_text}</del>'
                                                elif mark_type == "spoiler":
                                                    leaf_text = f'<span style="background-color: #000; color: #fff;">{leaf_text}</span>'
                                            content += leaf_text
                                content += '<br>'



                            elif node.get("type") == "block-quote-container":
                                block_quote_lines = node.get("nodes", [])
                                block_quote_content = ''
                                for block_quote_line in block_quote_lines:
                                    if block_quote_line.get("type") == "block-quote-line":
                                        block_quote_line_nodes = block_quote_line.get("nodes", [])
                                        for block_quote_line_node in block_quote_line_nodes:
                                            leaves = block_quote_line_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                block_quote_content += '<br>' + leaf_text.lstrip('>').strip()
                                content += f'<blockquote style="border-left: 4px solid #F5C400; padding: 4px;">{block_quote_content.strip()}</blockquote>'

                            elif node.get("type") == "unordered-list":
                                list_items = node.get("nodes", [])
                                unordered_list_content = ''
                                for list_item in list_items:
                                    if list_item.get("type") == "list-item":
                                        list_item_nodes = list_item.get("nodes", [])
                                        list_item_content = ''
                                        for list_item_node in list_item_nodes:
                                            leaves = list_item_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                list_item_content += leaf_text
                                        unordered_list_content += f'<li>{list_item_content}</li>'
                                content += f'<ul>{unordered_list_content}</ul>'
                            elif node.get("type") == "ordered-list":
                                list_items = node.get("nodes", [])
                                ordered_list_content = ''
                                for list_item in list_items:
                                    if list_item.get("type") == "list-item":
                                        list_item_nodes = list_item.get("nodes", [])
                                        list_item_content = ''
                                        for list_item_node in list_item_nodes:
                                            leaves = list_item_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                list_item_content += leaf_text
                                        ordered_list_content += f'<li>{list_item_content}</li>'
                                    elif list_item.get("type") == "ordered-list":
                                        nested_ordered_list_content = ''
                                        nested_list_items = list_item.get("nodes", [])
                                        for nested_list_item in nested_list_items:
                                            if nested_list_item.get("type") == "list-item":
                                                nested_list_item_nodes = nested_list_item.get("nodes", [])
                                                nested_list_item_content = ''
                                                for nested_list_item_node in nested_list_item_nodes:
                                                    nested_leaves = nested_list_item_node.get("leaves", [])
                                                    for nested_leaf in nested_leaves:
                                                        nested_leaf_text = nested_leaf.get("text", "")
                                                        nested_list_item_content += nested_leaf_text
                                                nested_ordered_list_content += f'<li>{nested_list_item_content}</li>'
                                        ordered_list_content += f'<ol>{nested_ordered_list_content}</ol>'
                                content += f'<ol>{ordered_list_content}</ol>'







            formatted_message = MESSAGE_TEMPLATE.format(
                avatar_url=avatar_url,
                created_by=created_by,
                created_at=created_at,
                content=content
            )
            formatted_messages.append(formatted_message)

        # Wait for all download tasks to complete
        for future in as_completed(futures):
            future.result()

    html = HTML_TEMPLATE.format(
        title=channel_data["name"],
        topic=channel_data.get("topic", ""),
        messages='\n'.join(formatted_messages),
        CSS=CSS
    )

    return html

# Process each channel
for channel in channels_data["channels"]:
    channel_dir = os.path.join(server_dir, channel["name"])
    for group in groups_data["groups"]:
        if channel["groupId"] == group["id"]:
            if verbose_mescount:
                print(f"Processing {group['name']} #{channel['name']}")
            group_dir = os.path.join(server_dir, group["name"] + " (" + group["id"] + ")")
            channel_dir = os.path.join(group_dir, channel["name"])
            break
    os.makedirs(channel_dir, exist_ok=True)

    # Create media folders
    os.makedirs(os.path.join(channel_dir, "media", "images"), exist_ok=True)
    os.makedirs(os.path.join(channel_dir, "media", "videos"), exist_ok=True)
    os.makedirs(os.path.join(channel_dir, "media", "files"), exist_ok=True)

    # Save channel info
    with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
        json.dump(channel, f, separators=(',', ':'))

    # Check if the channel's content type is "chat", "stream", or "voice"
    if channel.get("contentType") in ["chat", "stream", "voice"]:
        print(f"=========== FETCHING MESSAGES FROM #{channel['name']}...")
        messages = []
        beforeDate = None
        mesc = 0

        while True:
            params = {"limit": 100}
            if beforeDate:
                params["beforeDate"] = beforeDate

            response = fetch(f"channels/{channel['id']}/messages", params=params)
            messages.extend(response["messages"])

            mesc += len(response["messages"])
            #print(f"Collected {mesc} messages")

            if len(response["messages"]) < 100:
                break

            beforeDate = response["messages"][-1]["createdAt"]

        messages.reverse()

        # Save messages JSON
        with open(os.path.join(channel_dir, f"{channel['id']}_messages.json"), 'w') as f:
            json.dump(messages, f, separators=(',', ':'))

        if verbose_mescount:
            print(f"Found {len(messages)} messages")
            print(f"Saved all messages from #{channel['name']} to {channel_dir}/{channel['id']}_messages.json")


        # Generate and save HTML
        if len(messages) > messages_per_page:
            for i in range(0, len(messages), messages_per_page):
                html = generate_html(messages[i:i+messages_per_page], channel, members_json, server_dir, channel_dir)
                html_filename = f"{channel['name']} - Page {i//messages_per_page + 1}.html".replace('/', '-')
                with open(os.path.join(channel_dir, html_filename), 'w') as f:
                    f.write(html)
                if verbose_mescount:
                    print(f"Saved {channel_dir}/{html_filename}")
        else:
            html = generate_html(messages, channel, members_json, server_dir, channel_dir)
            html_filename = f"{channel['name']}.html".replace('/', '-')
            with open(os.path.join(channel_dir, html_filename), 'w') as f:
                f.write(html)
            if verbose_mescount:
                print(f"Saved {channel_dir}/{html_filename}")
        # # Generate and save HTML
        #html = generate_html(messages, channel, members_json, server_dir, channel_dir)
        #html_filename = f"{channel['name']}.html".replace('/', '-')
        #with open(os.path.join(channel_dir, html_filename), "w", encoding='utf-8') as f:
        #    f.write(html)
        #print(f"Saved HTML to {channel_dir}/{['name']}.html")


    if channel.get("contentType") == "forum":
        continue
        print(f"=========== FETCHING THREADS FROM #{channel['name']}...")
        forums = []
        print(f"Saved forum info to {channel_dir}/{channel['id']}_info.json")

        print(f"Fetching from endpoint: /api/{channel['id']}/forums") # Print the endpoint being used
        page = 1
        while True:
            response = fetch(f"api/{channel['id']}/forums", params={"maxItems": 30, "page": page})
            # **Check response status code**
            if response.status_code != 200:
                print(f"Error fetching data: {response.status_code} {response.json()}")
                break # Exit the loop if there's an error
            forums.extend(response["forums"])
            if not response["threads"]:
                break
            page += 1
        with open(os.path.join(channel_dir, f"{channel['id']}_forums.json"), 'w') as f:
           json.dump(forums, f, separators=(',', ':'))
        for forum in forums:
            fetch_channel(forum['id'])
            print(f"Saved forum info to {channel_dir}/{forum['id']}_info.json")
        for thread in response["threads"]:
            thread_id = thread["id"]
            thread_response = fetch(f"api/{channel['id']}/forums/{thread_id}")
            with open(os.path.join(channel_dir, f"{thread_id}_info.json"), 'w') as f:
                json.dump(thread_response, f, separators=(',', ':'))


    if channel.get("contentType") == "events":
        continue
        print(f"=========== FETCHING CALENDAR FROM #{channel['name']}...")
        events_dir = os.path.join(channel_dir, "events")
        os.makedirs(events_dir, exist_ok=True)
        params = { "endDate": "2024-08-12T04:00:00.000Z",
                   "maxItems": 250,
                   "startDate": "2024-06-24T04:00:00.000Z" }
        while True:
            events_data = fetch(f"channels/{channel['id']}/events", params=params)
            if not events_data:
                break



    if channel.get("contentType") == "schedule":
        continue


    if channel.get("contentType") == "media":
        continue
        print(f"=========== FETCHING MEDIA FROM #{channel['name']}...")
        media_dir = os.path.join(channel_dir, "media")
        os.makedirs(media_dir, exist_ok=True)
        media_data = fetch(f"channels/{channel['id']}/media")
        with open(os.path.join(channel_dir, f"{channel['id']}_media.json"), 'w') as f:
            json.dump(media_data, f, separators=(',', ':'))
        for media in media_data["media"]:
            media_url = media["url"]
            media_id = media["id"]
            media_filename = os.path.basename(media_url)
            media_filepath = os.path.join(media_dir, media_filename)
            if not os.path.exists(media_filepath):
                response = requests.get(media_url)
                if response.status_code == 200:
                    os.makedirs(os.path.dirname(media_filepath), exist_ok=True)
                    with open(media_filepath, "wb") as f:
                        f.write(response.content)
            post_data = fetch(f"channels/{channel['id']}/media/{media_id}")
            with open(os.path.join(channel_dir, f"{media_id}_post.json"), 'w') as f:
                json.dump(post_data, f, separators=(',', ':'))
        break


    if channel.get("contentType") == "doc":
        continue
        print(f"=========== FETCHING DOCS FROM #{channel['name']}...")
        docs_dir = os.path.join(channel_dir, "docs")
        os.makedirs(docs_dir, exist_ok=True)
        docs_data = fetch(f"channels/{channel['id']}/docs")
        with open(os.path.join(channel_dir, f"{channel['id']}_docs.json"), 'w') as f:
            json.dump(docs_data, f, separators=(',', ':'))
        for doc in docs_data["docs"]:
            doc_url = doc["title"]
            doc_id = doc["id"]
            doc_filename = os.path.basename(doc_url)
            doc_filepath = os.path.join(docs_dir, doc_filename)
            if not os.path.exists(doc_filepath):
                response = requests.get(doc_url)
                if response.status_code == 200:
                    os.makedirs(os.path.dirname(doc_filepath), exist_ok=True)
                    with open(doc_filepath, "wb") as f:
                        f.write(response.content)
        break


    if channel.get("contentType") == "announcement":
        continue
        print(f"=========== FETCHING ANNOUNCEMENTS FROM #{channel['name']}...")
        announcements_dir = os.path.join(channel_dir, "announcements")
        os.makedirs(announcements_dir, exist_ok=True)
        all_announcements = []
        params = {"maxItems": 5}
        while True:
            announcements_data = fetch(f"channels/{channel['id']}/announcements", params=params)
            if not announcements_data:
                break
            all_announcements.extend(announcements_data)
            if len(announcements_data) < params["maxItems"]:
                break
            params["start"] = announcements_data[-1]["id"]
        with open(os.path.join(channel_dir, f"{channel['id']}_announcements.json"), 'w') as f:
            json.dump(all_announcements, f, separators=(',', ':'))
        break


    if channel.get("contentType") == "list":
        continue
        print(f"=========== FETCHING TODO FROM #{channel['name']}...")
        todo_dir = os.path.join(channel_dir, "todo")
        os.makedirs(todo_dir, exist_ok=True)
        todo_data = fetch(f"channels/{channel['id']}/listitems")
        with open(os.path.join(channel_dir, f"{channel['id']}_list.json"), 'w') as f:
            json.dump(todo_data, f, separators=(',', ':'))
        break

for group in groups_data["groups"]:
    group_dir = os.path.join(server_dir, group["name"] + " (" + group["id"] + ")")
    if group.get("avatar"):
        parsed_url = urlparse(group["avatar"]) # Parse the URL
        pic_filename = os.path.basename(parsed_url.path) # Extract filename from parsed URL's path
        if not os.path.exists(os.path.join(group_dir, pic_filename)):
            os.makedirs(group_dir, exist_ok=True)
        pic_filepath = os.path.join(group_dir, pic_filename)
        response = requests.get(group["avatar"]) # Pass original URL string to requests.get
        if response.status_code == 200:
            with open(pic_filepath, "wb") as f:
                f.write(response.content)

print("Finished processing all channels.")

In [ ]:
# @title Download the Whole Server
import shutil
import subprocess
from google.colab import files
save_type = "7z" # @param ["7z", "targz", "zip"]
saving_to = "DirectDownLoad" # @param ["DirectDownLoad", "GoogleDrive"]
todl = f"{SAVE_DIRECTORY}{SERVER_NAME}"
if save_type == "7z":
    outfile = f"{SAVE_DIRECTORY}{SERVER_NAME}.7z"
    packer = subprocess.run(['7z', 'a', '-mx1', outfile, todl], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(packer.stdout.decode('utf-8'))
if save_type == "targz":
    outfile = f"{SAVE_DIRECTORY}{SERVER_NAME}.tar.gz"
    packer = subprocess.run(['tar', '-czvf', outfile, todl], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(packer.stdout.decode('utf-8'))
if save_type == "zip":
    outfile = f"{SAVE_DIRECTORY}{SERVER_NAME}.zip"
    packer = subprocess.run(['zip', '-r', outfile, todl], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(packer.stdout.decode('utf-8'))

if saving_to == "DirectDownLoad":
    files.download(outfile)
if saving_to == "GoogleDrive":
    drive.mount('/content/drive')
    shutil.copy(outfile, '/content/drive/MyDrive/')

# Direct Messages

In [ ]:
# @title Export all DMs and Media
messages_per_page = 15000 # @param {type:"integer"}
# @markdown Same deal as above, but you can probably increase the message count way more as DMs aren't as scary as servers. This also works on Groups.

# @markdown Make sure to at least tick the HTML File Setup cell if you skipped doing any server stuff.
verbose_dmfetch = False # @param {type:"boolean"}


def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "wb") as f:
            f.write(response.content)

def generate_html(messages, dm_data, members_data, dms_dir):
    formatted_messages = []
    doge = {}

    with ThreadPoolExecutor() as executor:
        futures = []

        for message in messages:
            created_by_id = message['createdBy']
            created_at = message.get('createdAt') or message.get('created_at')

            # Determine if it's a system message
            is_system_message = message.get('type') == 'system' or 'systemMessage' in str(message.get('content', {}))

            if is_system_message:
                member = next((m for m in dm_data['users'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"
                avatar_url = ""

                # Handle system message content
                content = ""
                document = message.get('content', {}).get('document', {})
                for node in document.get('nodes', []):
                    if node.get('type') == 'systemMessage':
                        data = node.get('data', {})
                        message_type = data.get('type')
                        if message_type == 'team-channel-created':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) created this chat channel."
                        elif message_type == 'channel-renamed':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) renamed this channel from '{data.get('oldName')}' to '{data.get('newName')}'."
                        elif message_type == 'streaming-screenshare-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started to share their screen."
                        elif message_type == 'voice-call-started':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) started a voice call."
                        elif message_type == 'webhookMessage':
                            embeds = node.get('data', {}).get('embeds', [])
                            content = json.dumps(data, indent=2)
                        else:
                            content = f"System action performed: {message_type}"


            else:
                # Find member data based on createdBy ID
                member = next((m for m in dm_data['users'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"
                #print("Member data:", member)  # Print the member data for inspection

                if member and member.get('profilePicture'):
                    profile_picture_url = member['profilePicture']
                    filename = f"{created_by_id}{os.path.splitext(os.path.basename(unquote(urlparse(profile_picture_url).path)))[1]}"
                    filepath = os.path.join(dm_dir, "media", filename)
                    if not os.path.exists(filepath):
                        os.makedirs(os.path.dirname(filepath), exist_ok=True)
                        with open(filepath, "wb") as f:
                            f.write(requests.get(profile_picture_url).content)
                    avatar_url = os.path.join("media", filename)
                else:
                    avatar_url = f"https://www.guilded.gg/asset/DefaultUserAvatars/profile_{doge.setdefault(created_by_id, (len(doge) % 5) + 1)}.png"


                # Process regular message content
                content = ""
                if "content" in message and message["content"]:
                    document = message["content"].get("document")
                    if document:
                        nodes = document.get("nodes", [])
                        for node in nodes:
                            if node.get("type") in ("image", "video", "fileUpload"):
                                file_src = node.get("data", {}).get("src", "")
                                if file_src:
                                    parsed_url = urlparse(file_src)
                                    media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                    if node.get("type") == "fileUpload":
                                        file_src = node.get("data", {}).get("src", "")
                                        if file_src:
                                            parsed_url = urlparse(file_src)
                                            media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                            #filename = f"{node.get('data', {}).get('name')}_{message['id']}"
                                            filename = f"{node.get('data', {}).get('name')}_{message['id']}{os.path.splitext(os.path.basename(unquote(urlparse(file_src).path)))[1]}"
                                            folder = "files"
                                            filepath = os.path.join(dm_dir, "media", folder, filename)
                                            os.makedirs(os.path.dirname(filepath), exist_ok=True)
                                            futures.append(executor.submit(download_file, media_url, filepath))



                                    elif node.get("type") == "image":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "images"
                                    elif node.get("type") == "video":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "videos"

                                    filepath = os.path.join(dm_dir, "media", folder, filename)
                                    futures.append(executor.submit(download_file, media_url, filepath))

                                    if node.get("type") == "image":
                                        content += f'<img src="{os.path.join("media", "images", filename)}">'
                                    elif node.get("type") == "video":
                                        content += f'<video controls><source src="{os.path.join("media", "videos", filename)}" type="video/webm">Your browser does not support the video tag.</video>'
                                    elif node.get("type") == "fileUpload":
                                        file_size_bytes = node.get("data", {}).get("fileSizeBytes")
                                        file_size_kb = round(file_size_bytes / 1024, 1)
                                        content += f'''
                                            <div class="file-upload-container">
                                                <div class="file-icon">
                                                    <svg class="icon icon-file-attach" shape-rendering="geometricPrecision" role="img">
                                                        <use xml:space="http://www.w3.org/1999/xlink" xlink:href="#icon-file-attach"></use>
                                                    </svg>
                                                </div>
                                                <div class="file-info">
                                                    <a href="{os.path.join("media", "files", filename)}" download class="file-name">{node.get('data', {}).get('name')}</a>
                                                    <div class="file-size">{file_size_kb} kb</div>
                                                </div>
                                                <a href="{os.path.join("media", filename)}" download class="download-icon">
                                                    <svg class="icon icon-download-tray" shape-rendering="geometricPrecision" role="img">
                                                        <use xml:space="http://www.w3.org/1999/xlink" xlink:href="#icon-download-tray"></use>
                                                    </svg>
                                                </a>
                                            </div>
                                        '''


                            if node.get("type") == "code-container":
                                code_lines = node.get("nodes", [])
                                code_content = ''
                                for code_line in code_lines:
                                    if code_line.get("type") == "code-line":
                                        code_line_nodes = code_line.get("nodes", [])
                                        for code_line_node in code_line_nodes:
                                            leaves = code_line_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                code_content += leaf_text + '\n'
                                language = node.get("data", {}).get("language", "")
                                if language:
                                    content += f'<pre><code class="language-{language}">{code_content.strip()}</code></pre>'
                                else:
                                    content += f'<pre><code>{code_content.strip()}</code></pre>'


                            elif node.get("type") == "paragraph":
                                sub_nodes = node.get("nodes", [])
                                for sub_node in sub_nodes:
                                    if sub_node.get("type") == "reaction":
                                        reaction_data = sub_node.get("data", {})
                                        if reaction_data and (reaction := reaction_data.get("reaction")) and (reaction_id := reaction.get("id")):
                                            if str(reaction_id).startswith("9000"):
                                                content += emoji.emojize(f":{reaction.get('name')}:", language='alias') if reaction.get('name') else ''
                                            else:

                                                custom_reaction = reaction.get("customReaction") or {}
                                                if isinstance(custom_reaction, bool):
                                                    custom_reaction = {}
                                                emote_url = custom_reaction.get("webp")
                                                emote_filename = custom_reaction.get("name")
                                                if emote_url and emote_filename:
                                                    emote_filepath = os.path.join(dms_dir, "media", "emotes", f"{emote_filename}.webp")
                                                    if not os.path.exists(emote_filepath):
                                                        try:
                                                            response = requests.get(emote_url)
                                                            if response.status_code == 200:
                                                                os.makedirs(os.path.dirname(emote_filepath), exist_ok=True)
                                                                with open(emote_filepath, "wb") as f:
                                                                    f.write(response.content)
                                                            else:
                                                                print(f"Failed to download custom reaction: {emote_filename} (Status code: {response.status_code})")
                                                        except requests.exceptions.RequestException as e:
                                                            print(f"Error downloading custom reaction: {emote_filename}")
                                                            print(e)
                                                            pass
                                                    content += f'<img src="{os.path.join("media", f"{emote_filename}.webp")}" alt="{emote_filename}" title="{emote_filename}" style="height: 1.3em; vertical-align: middle;">'


                                    elif sub_node.get("type") == "mention":
                                        mention_data = sub_node.get("data", {}).get("mention", {})
                                        mentioned_id = mention_data.get("id")
                                        mentioned_name = mention_data.get("name")
                                        display_name = f'@{mentioned_name}'
                                        content += display_name

                                    elif sub_node.get("type") == "link":
                                        link_data = sub_node.get("data", {})
                                        href = link_data.get("href", "")
                                        if href:
                                            link_text = sub_node.get("nodes", [{}])[0].get("leaves", [{}])[0].get("text", "")
                                            content += f'<a href="{href}">{link_text}</a>'

                                    elif sub_node.get("type") == "block-quote-container":
                                        block_quote_lines = sub_node.get("nodes", [])
                                        block_quote_content = ''
                                        for block_quote_line in block_quote_lines:
                                            if block_quote_line.get("type") == "block-quote-line":
                                                block_quote_line_nodes = block_quote_line.get("nodes", [])
                                                for block_quote_line_node in block_quote_line_nodes:
                                                    leaves = block_quote_line_node.get("leaves", [])
                                                    for leaf in leaves:
                                                        leaf_text = leaf.get("text", "")
                                                        marks = leaf.get("marks", [])
                                                        for mark in marks:
                                                            mark_type = mark.get("type")
                                                            if mark_type == "inline-code-v2":
                                                                leaf_text = f'<code>{leaf_text}</code>'
                                                            elif mark_type == "bold":
                                                                leaf_text = f'<strong>{leaf_text}</strong>'
                                                            elif mark_type == "italic":
                                                                leaf_text = f'<em>{leaf_text}</em>'
                                                            elif mark_type == "underline":
                                                                leaf_text = f'<u>{leaf_text}</u>'
                                                            elif mark_type == "strikethrough":
                                                                leaf_text = f'<del>{leaf_text}</del>'
                                                            elif mark_type == "spoiler":
                                                                leaf_text = f'<span style="background-color: #000; color: #fff;">{leaf_text}</span>'
                                                        block_quote_content += leaf_text
                                                block_quote_content += '<br>'
                                        content += f'<div style="background-color: #f0f0f0; border-left: 4px solid #F5C400; padding: 10px;">{block_quote_content}</div>'


                                    else:
                                        leaves = sub_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            marks = leaf.get("marks", [])
                                            for mark in marks:
                                                mark_type = mark.get("type")
                                                if mark_type == "inline-code-v2":
                                                    leaf_text = f'<code>{leaf_text}</code>'
                                                elif mark_type == "bold":
                                                    leaf_text = f'<strong>{leaf_text}</strong>'
                                                elif mark_type == "italic":
                                                    leaf_text = f'<em>{leaf_text}</em>'
                                                elif mark_type == "underline":
                                                    leaf_text = f'<u>{leaf_text}</u>'
                                                elif mark_type == "strikethrough":
                                                    leaf_text = f'<del>{leaf_text}</del>'
                                            content += leaf_text
                                content += '<br>'

                            elif node.get("type") == "block-quote-container":
                                block_quote_lines = node.get("nodes", [])
                                block_quote_content = ''
                                for block_quote_line in block_quote_lines:
                                    if block_quote_line.get("type") == "block-quote-line":
                                        block_quote_line_nodes = block_quote_line.get("nodes", [])
                                        for block_quote_line_node in block_quote_line_nodes:
                                            leaves = block_quote_line_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                block_quote_content += '<br>' + leaf_text.lstrip('>').strip()
                                content += f'<blockquote style="border-left: 4px solid #F5C400; padding: 4px;">{block_quote_content.strip()}</blockquote>'

                            elif node.get("type") == "unordered-list":
                                list_items = node.get("nodes", [])
                                unordered_list_content = ''
                                for list_item in list_items:
                                    if list_item.get("type") == "list-item":
                                        list_item_nodes = list_item.get("nodes", [])
                                        list_item_content = ''
                                        for list_item_node in list_item_nodes:
                                            leaves = list_item_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                list_item_content += leaf_text
                                        unordered_list_content += f'<li>{list_item_content}</li>'
                                content += f'<ul>{unordered_list_content}</ul>'
                            elif node.get("type") == "ordered-list":
                                list_items = node.get("nodes", [])
                                ordered_list_content = ''
                                for list_item in list_items:
                                    if list_item.get("type") == "list-item":
                                        list_item_nodes = list_item.get("nodes", [])
                                        list_item_content = ''
                                        for list_item_node in list_item_nodes:
                                            leaves = list_item_node.get("leaves", [])
                                            for leaf in leaves:
                                                leaf_text = leaf.get("text", "")
                                                list_item_content += leaf_text
                                        ordered_list_content += f'<li>{list_item_content}</li>'
                                    elif list_item.get("type") == "ordered-list":
                                        nested_ordered_list_content = ''
                                        nested_list_items = list_item.get("nodes", [])
                                        for nested_list_item in nested_list_items:
                                            if nested_list_item.get("type") == "list-item":
                                                nested_list_item_nodes = nested_list_item.get("nodes", [])
                                                nested_list_item_content = ''
                                                for nested_list_item_node in nested_list_item_nodes:
                                                    nested_leaves = nested_list_item_node.get("leaves", [])
                                                    for nested_leaf in nested_leaves:
                                                        nested_leaf_text = nested_leaf.get("text", "")
                                                        nested_list_item_content += nested_leaf_text
                                                nested_ordered_list_content += f'<li>{nested_list_item_content}</li>'
                                        ordered_list_content += f'<ol>{nested_ordered_list_content}</ol>'
                                content += f'<ol>{ordered_list_content}</ol>'


            formatted_message = MESSAGE_TEMPLATE.format(
                avatar_url=avatar_url,
                created_by=created_by,
                created_at=created_at,
                content=content
            )
            formatted_messages.append(formatted_message)

        # Wait for all download tasks to complete
        for future in as_completed(futures):
            future.result()

    html = HTML_TEMPLATE.format(
        title=dm_data["name"],
        topic=", ".join(member["name"] for member in dm_data["users"]),
        messages='\n'.join(formatted_messages),
        CSS=CSS
    )

    return html



myself_data = fetch_myself()
with open(os.path.join(SAVE_DIRECTORY, "myself.json"), "w") as f:
    f.write(json.dumps(myself_data, separators=(',', ':')))
    print(f"Saved {SAVE_DIRECTORY}myself.json")
USER_ID = myself_data["user"]["id"]
dms_data = fetch_dms()
with open(os.path.join(SAVE_DIRECTORY, "dms.json"), "w") as f:
    f.write(json.dumps(dms_data, separators=(',', ':')))
    print(f"Saved {SAVE_DIRECTORY}dms.json")


for channel in dms_data['channels']:
    channel_id = channel['id']
    #print(f"Channel ID: {channel_id}")
    dm_data = fetch_channel(channel_id)
    #print(f"Channel Data: {dm_data}")
    partner = next((user for user in channel['users'] if user['id'] != USER_ID), None)
    if channel.get("dmType") == "Group":
        user_names = [user["name"] for user in channel["users"]]
        dm_foldername = f"{channel_id} - {', '.join(user_names[:20])}"[:255]
    elif partner:
        dm_foldername = f"{partner['name']} ({partner['id']})"
    else:
        dm_foldername = f"[deleted user] ({channel_id})"
    dm_dir = os.path.join(SAVE_DIRECTORY, "Direct Messages", sanitize_filename(dm_foldername))
    os.makedirs(dm_dir, exist_ok=True)
    with open(os.path.join(dm_dir, f"{channel_id}_info.json"), 'w') as f:
        json.dump(dm_data, f, separators=(',', ':'))
        if verbose_dmfetch:
            print(f"Saving channel {channel_id} for user {partner['name'] if partner else 'Unknown'}")
    members_dict = {}
    for member in dm_data['users']:
        members_dict[member['name']] = member

    url = f"https://www.guilded.gg/api/channels/{channel_id}/messages"
    beforeDate = None # Reset beforeDate for each channel
    messages = [] # Reset messages for each channel
    mesc = 0 # Reset mesc for each channel
    while True:
        params = {"maxItems": 100, "beforeDate": beforeDate}
        response = requests.get(url, params=params, cookies=cookies)
        dm_mes = unshid_cdn(response.json())
        #response = fetch(f"api/channels/{channel_id}/messages", params={"maxItems": 100, "beforeDate": beforeDate})
        messages.extend(dm_mes["messages"])
        mesc += len(dm_mes["messages"])
        if verbose_dmfetch:
            print(f"Collected {mesc} messages")
        if len(dm_mes["messages"]) < 100:
            break
        beforeDate = dm_mes["messages"][-1]["createdAt"]
    messages.reverse()
    # Save messages JSON
    with open(os.path.join(dm_dir, f"{channel_id}_messages.json"), 'w') as f:
        json.dump(messages, f, separators=(',', ':'))
    if verbose_dmfetch:
        print(f"Found {len(messages)} messages")

    # Generate and save HTML
    if len(messages) > messages_per_page:
        for i in range(0, len(messages), messages_per_page):
            html = generate_html(messages[i:i+messages_per_page], dm_data, members_dict, dm_dir)
            html_filename = f"DMs with {partner['name']} - Page {i//messages_per_page + 1}.html".replace('/', '-')
            if channel.get("dmType") == "Group":
                html_filename = f"Group {channel_id} - Page {i//messages_per_page + 1}.html".replace('/', '-')
            with open(os.path.join(dm_dir, html_filename), 'w') as f:
                f.write(html)
            if verbose_dmfetch:
                print(f"Saved {dm_dir}/{html_filename}")
    else:
        html = generate_html(messages, dm_data, members_dict, dm_dir)
        html_filename = f"DMs with {partner['name']}.html".replace('/', '-')
        if channel.get("dmType") == "Group":
            html_filename = f"Group {channel_id}.html".replace('/', '-')
        with open(os.path.join(dm_dir, html_filename), 'w') as f:
            f.write(html)
        if verbose_dmfetch:
            print(f"Saved {dm_dir}/{html_filename}")
    ## Generate and save HTML
    #html = generate_html(messages, dm_data, members_dict, dm_dir)
    #html_filename = f"DMs with {partner['name']}.html".replace('/', '-')
    #if channel.get("dmType") == "Group":
    #    html_filename = f"Group {channel_id}.html".replace('/', '-')
    #with open(os.path.join(dm_dir, html_filename), "w", encoding='utf-8') as f:
    #    f.write(html)


    print(f"Saved HTML to {dm_dir}/{partner['name']}.html")

print("Finished processing all DMs.")

In [ ]:
# @title Download and Save DMs (dont ask why it's different here)
save_type = "7z" # @param ["7z", "targz", "zip"]
saving_to = "DirectDownLoad" # @param ["DirectDownLoad", "GoogleDrive"]
from google.colab import drive

todl = SAVE_DIRECTORY + "Direct Messages"
if save_type == "7z":
    outfile = f"{SAVE_DIRECTORY}Direct_Messages.7z"
    !7z a {outfile} "{todl}"
if save_type == "targz":
    outfile = f"{SAVE_DIRECTORY}Direct_Messages.tar.gz"
    !tar -czvf {outfile} "{todl}"
if save_type == "zip":
    outfile = f"{SAVE_DIRECTORY}Direct_Messages.zip"
    !zip -r {outfile} "{todl}"

if saving_to == "DirectDownLoad":
    files.download(outfile)
if saving_to == "GoogleDrive":
    drive.mount('/content/drive')
    shutil.copy(outfile, '/content/drive/MyDrive/')

#if save_to_gdrive:
    #shutil.copy(SERVER_NAME + '.zip', '/content/drive/MyDrive/' + SERVER_NAME + '.zip')
    #print("Copied zip to Google Drive.")


In [ ]:
# @title (Debug) Preview HTML (wont show local files)
# @markdown You can test how messages look. But you have to put the link manually, I changed how folders work too much
from IPython.display import HTML

directory = "" # @param {type:"string"}

with open(directory, "r") as f:
    html_content = f.read()
display(HTML(html_content))
